In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
        
    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i']) 
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)

    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i'])
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    
    if PARAM['atmpro'] == PARAM['atmpro']:
        atmpro = PARAM['atmpro']
    else:
        atmpro = 'mls'
        
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = 523cbb7
molecule = {'ch4': 1.8e-06}
band = [9]
atmpro = mls
tsfc = 294


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o ch4 band7
  atmpro = mls
  band = 7
  commitnumber = a06b618
  conc = 1.8e-06
  dv = 0.001
  klin = 2e-21
  molecule = ch4
  ng_adju = [0, 0]
  ng_refs = [2, 3]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (500, 250)]
  tsfc = 294
  vmax = 1380
  vmin = 1215
  w_diffuse = [(1.66, 1.66), (1.66, 1.66, 1.66)]
  wgt = [(0.7, 0.7), (0.4, 0.6, 0.75)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1 -0.103256                    1 -0.062917
0.000750        2 -0.102576                    2 -0.062443
0.001052        3 -0.101012                    3 -0.061420
0.001476        4 -0.099346                    4 -0.060309
0.002070        5 -0.097681                    5 -0.059219
0.002904        6 -0.096051                    6 -0.058071
0.004074        7 -0.094462                    7 -0.056951
0.005714        8 -0.092941                    8 -0.055771
0.008015        9 -0.091520                    9 -0.054631
0.011243       10 -0.090253                   10 -0.053434
0.015771       11 -0.089215                   11 -0.052252
0.022122       12 -0.088495                   12 -0.051107
0.031031       13 -0.088230                   13 -0.049964
0.043528       14 -0.088501                   14 -0.048818
0.061057       15 -0.085147                   15 -0.044784
0.085645       16 -0.069571                   16 -0.032137
0.120136       17 -0.041543                   17 -0.010375
0.168516       18 -0.006824                   18  0.017340
0.236378       19  0.034932                   19  0.051534
0.331549       20  0.084869                   20  0.093383
0.465100       21  0.144491                   21  0.144260
0.652400       22  0.215988                   22  0.205037
0.915100       23  0.279324                   23  0.260000
1.283650       24  0.269195                   24  0.260183
1.800600       25  0.196056                   25  0.208064
2.525700       26  0.126686                   26  0.149410
3.542800       27  0.076681                   27  0.098218
4.969550       28  0.044832                   28  0.058215
6.970850       29  0.027253                   29  0.031328
9.778100       30  0.018033                   30  0.016033
13.715850      31  0.011998                   31  0.008452
19.239350      32  0.007031                   32  0.004998
26.987250      33  0.003814                   33  0.003901
37.855300      34  0.001530                   34  0.002687
53.100050      35 -0.002111                   35 -0.001060
73.887500      36 -0.007288                   36 -0.007055
97.662500      37 -0.011910                   37 -0.012638
121.437500     38 -0.015828                   38 -0.017547
145.212500     39 -0.021121                   39 -0.023565
168.987500     40 -0.026877                   40 -0.029560
192.762500     41 -0.029327                   41 -0.032119
216.537500     42 -0.027948                   42 -0.030959
240.312500     43 -0.026171                   43 -0.028985
264.087500     44 -0.024502                   44 -0.027020
287.862500     45 -0.022747                   45 -0.024973
311.637500     46 -0.020828                   46 -0.022812
335.412500     47 -0.018711                   47 -0.020496
359.187500     48 -0.016389                   48 -0.018078
382.962500     49 -0.013865                   49 -0.015494
406.737500     50 -0.011169                   50 -0.012800
430.512500     51 -0.008370                   51 -0.010070
454.287500     52 -0.005549                   52 -0.007378
478.062500     53 -0.002756                   53 -0.004661
501.837500     54  0.000022                   54 -0.002007
525.612500     55  0.002842                   55  0.000672
549.387500     56  0.005755                   56  0.003463
573.162500     57  0.008795                   57  0.006403
596.937500     58  0.011982                   58  0.009517
620.712500     59  0.015333                   59  0.012818
644.487500     60  0.018861                   60  0.016338
668.262500     61  0.022585                   61  0.020101
692.037500     62  0.026532                   62  0.024068
715.812500     63  0.030731                   63  0.028396
739.587500     64  0.035215                   64  0.033002
763.362500     65  0.040000                   65  0.037857
78

# Fluxes by Layer

CRD                          CLIRAD                \
                        flug      fldg      fnetg       flug          fldg   
pressure    level                                                            
0.000000    1     -19.274965  0.000000 -19.274965 -19.303396  8.920823e-08   
0.000624    2     -19.274975  0.000003 -19.274973 -19.303403  1.845531e-06   
0.000876    3     -19.274980  0.000004 -19.274976 -19.303405  2.561563e-06   
0.001229    4     -19.274985  0.000005 -19.274980 -19.303409  3.587899e-06   
0.001723    5     -19.274993  0.000008 -19.274986 -19.303414  5.060786e-06   
0.002417    6     -19.275005  0.000011 -19.274994 -19.303421  7.172420e-06   
0.003391    7     -19.275020  0.000015 -19.275005 -19.303431  1.020248e-05   
0.004757    8     -19.275042  0.000022 -19.275020 -19.303444  1.454682e-05   
0.006672    9     -19.275073  0.000031 -19.275041 -19.303463  2.077970e-05   
0.009359    10    -19.275115  0.000045 -19.275070 -19.303489  2.971318e-05   
0.013128    11    -19.275175  0.000064 -19.275111 -19.303526  4.252640e-05   
0.018415    12    -19.275258  0.000091 -19.275167 -19.303577  6.089469e-05   
0.025830    13    -19.275374  0.000130 -19.275244 -19.303648  8.720654e-05   
0.036232    14    -19.275538  0.000185 -19.275353 -19.303748  1.248959e-04   
0.050823    15    -19.275768  0.000262 -19.275506 -19.303886  1.788968e-04   
0.071291    16    -19.276089  0.000377 -19.275712 -19.304076  2.598779e-04   
0.100000    17    -19.276516  0.000567 -19.275949 -19.304321  3.957851e-04   
0.140271    18    -19.277051  0.000904 -19.276147 -19.304614  6.395926e-04   
0.196760    19    -19.277688  0.001495 -19.276193 -19.304934  1.075477e-03   
0.275997    20    -19.278384  0.002519 -19.275865 -19.305222  1.846974e-03   
0.387100    21    -19.279010  0.004262 -19.274748 -19.305343  3.197728e-03   
0.543100    22    -19.279266  0.007188 -19.272078 -19.305019  5.539714e-03   
0.761700    23    -19.278482  0.011997 -19.266485 -19.303703  9.533009e-03   
1.068500    24    -19.275686  0.019352 -19.256334 -19.300658  1.593835e-02   
1.498800    25    -19.270967  0.028355 -19.242612 -19.295955  2.449900e-02   
2.102400    26    -19.265523  0.036929 -19.228594 -19.290485  3.390675e-02   
2.949000    27    -19.259909  0.044020 -19.215889 -19.284687  4.309432e-02   
4.136600    28    -19.254393  0.049291 -19.205102 -19.278801  5.102707e-02   
5.802500    29    -19.249190  0.052935 -19.196255 -19.273028  5.674321e-02   
8.139200    30    -19.244392  0.055681 -19.188711 -19.267469  5.985657e-02   
11.417000   31    -19.240113  0.058403 -19.181709 -19.262267  6.088121e-02   
16.014700   32    -19.236931  0.061756 -19.175175 -19.258053  6.127096e-02   
22.464000   33    -19.236181  0.066378 -19.169803 -19.256181  6.321769e-02   
31.510500   34    -19.239343  0.073627 -19.165716 -19.258291  6.950755e-02   
44.200100   35    -19.248398  0.084981 -19.163416 -19.266844  8.210031e-02   
62.000000   36    -19.268651  0.100783 -19.167868 -19.288098  1.011202e-01   
85.775000   37    -19.309061  0.120669 -19.188392 -19.332412  1.255617e-01   
109.550000  38    -19.362820  0.140886 -19.221934 -19.392510  1.500621e-01   
133.325000  39    -19.428978  0.162466 -19.266512 -19.467449  1.755778e-01   
157.100000  40    -19.510215  0.184220 -19.325995 -19.559327  2.010813e-01   
180.875000  41    -19.609481  0.207792 -19.401689 -19.669268  2.277636e-01   
204.650000  42    -19.723933  0.239651 -19.484283 -19.793138  2.611669e-01   
228.425000  43    -19.847545  0.284553 -19.562992 -19.925195  3.060261e-01   
252.200000  44    -19.978041  0.341343 -19.636699 -20.063090  3.622814e-01   
275.975000  45    -20.114452  0.408748 -19.705705 -20.205920  4.290075e-01   
299.750000  46    -20.255932  0.486163 -19.769769 -20.352840  5.055900e-01   
323.525000  47    -20.401667  0.573241 -19.828426 -20.502954  5.914514e-01   
347.300000  48    -20.550861  0.669737 -19.881124 -20.655574  6.863409e-01   
371.075000  49    -20.702725  0.775445 -19.9272

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')